In [1]:
import yaml
import fitz
import torch
import gradio as gr
from PIL import Image
from langchain import hub
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
#from langchain_community.llms import HuggingFacePipeline
#from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel
#TEST
from FakeLLM import FakePromptCopyLLM

d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pdf_loader = PyPDFLoader("../documents/barlowtwins-CXR.pdf")
documents = pdf_loader.load()

In [6]:
documents

[Document(page_content='BarlowTwins-CXR: Enhancing Chest X-Ray\nBased Abnormality Localization with\nSelf-Supervised Learning\nHaoyue Sheng1,2,3*, Linrui Ma1,2, Jean-Fran¸ cois Samson3,\nDianbo Liu2,4\n1*D´ epartement d’informatique et de recherche op´ erationnelle, Universit´ e\nde Montr´ eal, 2920 chemin de la Tour, Montr´ eal, H3T 1J4, QC, Canada.\n2Mila - Quebec AI Institute, 6666 Rue Saint-Urbain, Montr´ eal, H2S\n3H1, QC, Canada.\n3Direction des ressources informationnelles, CIUSSS du\nCentre-Sud-de-l’ ˆIle-de-Montr´ eal, 400 Blvd. De Maisonneuve Ouest,\nMontr´ eal, H3A 1L4, QC, Canada.\n4School of Medicine and College of Design and Engineering, National\nUniversity of Singapore, 21 Lower Kent Ridge Rd, Singapore, 119077,\nSG, Singapore.\n*Corresponding author(s). E-mail(s): haoyue.sheng@umontreal.ca;\nContributing authors: linrui.ma@umontreal.ca;\njean-francois.samson.ccsmtl@ssss.gouv.qc.ca; dianbo@nus.edu.sg;\nAbstract\nBackground: Chest X-ray imaging based abnormality localiza

In [4]:
import re  # For regular expressions
from nltk.corpus import stopwords  # You'll need to install NLTK first
import copy
def clean_text(text):
  text = text.lower()  # Convert to lowercase
  text = re.sub(r'[^a-z0-9\s-]', '', text)  # Remove non-alphanumeric characters (except space and dash)
  text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single spaces
  return text

def remove_stop_words(text):
  stop_words = set(stopwords.words('english'))
  return " ".join([word for word in text.split() if word not in stop_words])

def preprocess_documents(documents):
  for doc in documents:
    # Apply basic cleaning
    cleaned_text = clean_text(doc.page_content)
    # Optionally remove stop words
    # cleaned_text = remove_stop_words(cleaned_text)  # Uncomment if you want stop word removal
    doc.page_content = cleaned_text
  return documents
documents_copy = copy.deepcopy(documents)
preprocessed_documents = preprocess_documents(documents_copy)
preprocessed_documents

[Document(page_content='barlowtwins-cxr enhancing chest x-ray based abnormality localization with self-supervised learning haoyue sheng123 linrui ma12 jean-fran cois samson3 dianbo liu24 1d epartement dinformatique et de recherche op erationnelle universit e de montr eal 2920 chemin de la tour montr eal h3t 1j4 qc canada 2mila - quebec ai institute 6666 rue saint-urbain montr eal h2s 3h1 qc canada 3direction des ressources informationnelles ciusss du centre-sud-de-l ile-de-montr eal 400 blvd de maisonneuve ouest montr eal h3a 1l4 qc canada 4school of medicine and college of design and engineering national university of singapore 21 lower kent ridge rd singapore 119077 sg singapore corresponding authors e-mails haoyueshengumontrealca contributing authors linruimaumontrealca jean-francoissamsonccsmtlssssgouvqcca dianbonusedusg abstract background chest x-ray imaging based abnormality localization essential in diagnosing various diseases faces significant clinical challenges due to comple

In [5]:
documents_copy = copy.deepcopy(documents)
preprocessed_documents = preprocess_documents(documents_copy)
preprocessed_documents

[Document(page_content='barlowtwins-cxr enhancing chest x-ray based abnormality localization with self-supervised learning haoyue sheng123 linrui ma12 jean-fran cois samson3 dianbo liu24 1d epartement dinformatique et de recherche op erationnelle universit e de montr eal 2920 chemin de la tour montr eal h3t 1j4 qc canada 2mila - quebec ai institute 6666 rue saint-urbain montr eal h2s 3h1 qc canada 3direction des ressources informationnelles ciusss du centre-sud-de-l ile-de-montr eal 400 blvd de maisonneuve ouest montr eal h3a 1l4 qc canada 4school of medicine and college of design and engineering national university of singapore 21 lower kent ridge rd singapore 119077 sg singapore corresponding authors e-mails haoyueshengumontrealca contributing authors linruimaumontrealca jean-francoissamsonccsmtlssssgouvqcca dianbonusedusg abstract background chest x-ray imaging based abnormality localization essential in diagnosing various diseases faces significant clinical challenges due to comple

In [19]:
def make_retriever(doc):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
    all_splits = text_splitter.split_documents(doc)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    doc = Chroma.from_documents(all_splits, embeddings)
    retriever=doc.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    return retriever
retriever_org = make_retriever(documents)


In [20]:
retriever_org.invoke("What is the main topic of the document?")

[Document(page_content='methodology analyzed data was responsible for experiments and results visualization and participated in manuscript drafting and revision lm assisted in developing the research methodology and contributed to the drafting and revision of the manuscript jfs collected and interpreted data and provided expertise in statistical analysis dl contributed to the study design offered statistical analysis expertise assisted in interpreting results and played a significant role in the critical revision of the manuscript all authors read and approved the final manuscript 97 acknowledgements the authors wish to express their gratitude to ciusss du centre-sud-de-l le-de- montr eal for the computational resources and support provided which were essential for the research conducted as part of the graduate internship program we are espe- cially thankful to our department director mathieu mailhot for his mentorship and to chen cheng for his collaborative efforts and valuable contri

In [16]:
retriever_pre = make_retriever(preprocessed_documents)

In [17]:
retriever_pre.invoke("What is the main topic of the document?")

[Document(page_content='methodology analyzed data was responsible for experiments and results visualization and participated in manuscript drafting and revision lm assisted in developing the research methodology and contributed to the drafting and revision of the manuscript jfs collected and interpreted data and provided expertise in statistical analysis dl contributed to the study design offered statistical analysis expertise assisted in interpreting results and played a significant role in the critical revision of the manuscript all authors read and approved the final manuscript 97 acknowledgements the authors wish to express their gratitude to ciusss du centre-sud-de-l le-de- montr eal for the computational resources and support provided which were essential for the research conducted as part of the graduate internship program we are espe- cially thankful to our department director mathieu mailhot for his mentorship and to chen cheng for his collaborative efforts and valuable contri